# Conversational Agent with Context Awareness

In [1]:
# %pip install -q langchain langchain_experimental openai python-dotenv langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Now get the API key (it should be loaded from .env file)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
else:
    print("Warning: OPENAI_API_KEY not found in environment variables.")
    print("Please make sure you have a .env file with OPENAI_API_KEY=your_api_key")

### Initialize language model

In [3]:
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### in-memory store for chat histories

In [4]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

### Prompt template

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

### Combine the prompt and model into a runnable chain

In [6]:
chain = prompt | llm

### Wrap the chain with message history

In [7]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

### Example usage


In [8]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

AI: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
AI: Your previous message was, "Hello! How are you?" How can I assist you further?


### Print the conversation history

In [9]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you?
ai: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
human: What was my previous message?
ai: Your previous message was, "Hello! How are you?" How can I assist you further?
